<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Выводы-по-предобработке:" data-toc-modified-id="Выводы-по-предобработке:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Выводы по предобработке:</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [15]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords as nltk_stopwords

import re
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings('ignore')
m = Mystem()

from tqdm import tqdm
tqdm.pandas()

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv('toxic_comments.csv')

In [3]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [6]:
#Проверим количество дубликатов
df.duplicated().sum()

0

In [7]:
#Проверим соотношение классов
df.value_counts('toxic')

toxic
0    143106
1     16186
dtype: int64

Наблюдается ясный перекос классов. Для того, чтобы получить более высокое значение F1-меры, выполним взвешивание классов.

In [3]:
#Очистим текст от лишних символов
def cleaning(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    text = text.lower()
    return text

df['text'] = df['text'].apply(cleaning)

In [23]:
wnl = WordNetLemmatizer()

def penn2morphy(penntag):
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

def lemmatize(text): 
    return " ".join([wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
                     for word, tag in pos_tag(word_tokenize(text))])

In [24]:
df['lemm_text'] = df['text'].progress_apply(lemmatize)

100%|██████████| 159292/159292 [09:11<00:00, 288.70it/s]


Функция, выполняющая лемматизацию работает очень долго. Не понимаю как выполнить ее, не потратив 2 дня на это. Для того, чтобы выполнить проект с BERT, необходим файл со словарем, как я понял из темы. Файла приложено не было. Так что оставляю так как есть.

In [28]:
stopwords = set(nltk_stopwords.words('english'))

### Выводы по предобработке:
Пропусков не обнаружено;

Дубликатов не обнаружено;

Наблюдается явный перекос классов. Нужно будет учесть это при обучении модели;

## Обучение

In [26]:
#Разделим модели на тренировочную и тестовую выборки. Результат будем проверять на кросс - валидации.
features = df['lemm_text']
target = df['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, shuffle=False, test_size=0.25, random_state = 42)

In [29]:
#Превратим текст в векторный вид
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tfidf_train = count_tf_idf.fit_transform(features_train)
tfidf_test = count_tf_idf.transform(features_test)

### Случайный лес

Код ниже лучше не запускать, иначе можно потерять 40 минут времени. Почему - то лучший счет в содели случайного леса - 0))

In [32]:
%%time
params_rf = {
    'n_estimators': list(range(50,300,50)),
    'max_depth':[5,15],
    'max_features' : list(range(1,20, 2))
}


rf = RandomForestClassifier(class_weight = 'balanced',  
                            random_state=42)
                                 
grid_rf = GridSearchCV(rf, param_grid=params_rf, scoring='f1', cv=3, verbose=True, n_jobs=-1)
best_grid = grid_rf.fit(tfidf_train, target_train)
print('F1 score:', grid_rf.best_score_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
F1 score: 0.32472834356332253
CPU times: total: 21.3 s
Wall time: 50min 57s


### Логистическая регрессия

In [31]:
param_grid = {
    'penalty' : ['l1', 'l2'],
    'C': list(range(1,15,3))
}

lr = LogisticRegression(class_weight = 'balanced', 
                        random_state=42,
                        max_iter=200)

grid_lr = GridSearchCV(lr, param_grid=param_grid, scoring='f1', cv=3, verbose=True, n_jobs=-1)
best_grid = grid_lr.fit(tfidf_train, target_train)
print('F1 score:', grid_lr.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
F1 score: 0.7587194472562331


### Тестирование лучшей модели

In [34]:
lr = LogisticRegression(
    penalty = grid_lr.best_params_['penalty'],
    C = grid_lr.best_params_['C'],
    max_iter=200,
    class_weight = 'balanced',
    random_state=42)

lr.fit(tfidf_train, target_train)

f1_score(target_test, lr.predict(tfidf_test))

0.7604456824512537

## Выводы

При обучении моделей был учтен дисбаланс классов. Из рассмотренных моделей, лучшее качество показала логистическая регрессия.  Значение F1 меры на тестовой выборке удовлетворяет условиям задачи. 